In [1]:
from keras.callbacks import ModelCheckpoint#, TensorBoard, EarlyStopping
from multiprocessing import Process
import LoadBatches
#from keras import optimizers
import math
import glob
import tensorflow as tf
from transunet import TransUNet
from keras import backend as K
from keras.losses import binary_crossentropy
from Models import UNet,DeeplabV3Plus,SegNet3,DLinkNet

import os

C:\Users\ranad\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\ranad\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\ranad\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0

In [2]:
def generalized_dice_coefficient(y_true, y_pred):
        smooth = 1.
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        score = (2. * intersection + smooth) / (
                    K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
        return score

In [3]:
def dice_loss(y_true, y_pred):
        loss = 1 - generalized_dice_coefficient(y_true, y_pred)
        return loss

In [4]:
def bce_dice_loss( y_true, y_pred):
        loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
        return loss / 2.0

In [6]:
import shutil

# Define source directory and destination directory
source_dir = 'dataset/train/palsar/'
destination_dir = 'dataset/train/palsar/image'

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Traverse files in the source directory
for filename in os.listdir(source_dir):
    # Check if the filename contains "_mask"
    if "_sat" in filename:
        # Form the full path for the source file
        source_file = os.path.join(source_dir, filename)
        
        # Form the full path for the destination file
        destination_file = os.path.join(destination_dir, filename)
        
        # Move the file to the destination directory
        shutil.move(source_file, destination_file)

print("Images containing '_sat' in their names have been moved to 'image' folder.")

Images containing '_sat' in their names have been moved to 'image' folder.


In [7]:
key = "funet"

train_batch_size = 4
val_batch_size = 1

n_classes = 2
epochs = 10000

input_height = 256
input_width = 256

loss = bce_dice_loss

i=0

train_images_path="dataset/train/palsar/image/"
train_segs_path="dataset/train/palsar/label/"

val_images_path = "dataset/test/palsar/image/"
val_segs_path = "dataset/test/palsar/label/"

model_path="output/"

tmp= sorted(glob.glob(train_images_path + "*.jpg") + glob.glob(train_images_path + "*.tif") +
            glob.glob(train_images_path + "*.png") + glob.glob(train_images_path + "*.jpeg"))

img_num=len(tmp)

tmp= sorted(glob.glob(val_images_path + "*.jpg") + glob.glob(val_images_path + "*.tif") +
            glob.glob(val_images_path + "*.png") + glob.glob(val_images_path + "*.jpeg"))

val_num=len(tmp)

In [8]:
val_num

776

In [9]:
img_num

3101

In [ ]:
m = DLinkNet.dlinknet() #SegNet3.SegNet(256,256,2)#input_height,input_width,n_classes)

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
   
m.compile(
    loss='binary_crossentropy',#loss,#'binary_crossentropy', # or loss=loss
    optimizer=opt,
    metrics=[tf.keras.metrics.BinaryAccuracy()]  #binary_accuracy
)

train_set = LoadBatches.imageSegmentationGenerator(train_images_path,
                                           train_segs_path, train_batch_size, n_classes=n_classes, input_height=input_height, input_width=input_width)
val_set = LoadBatches.imageSegmentationGenerator(val_images_path,
                                            val_segs_path, val_batch_size, n_classes=n_classes, input_height=input_height, input_width=input_width)

checkpoint = ModelCheckpoint(
    filepath="output/net.hdf5",
    monitor='val_loss',
    mode='min',
    save_best_only=False,
    save_weights_only=False
    )

m.fit(
    x=train_set,
    steps_per_epoch=img_num // train_batch_size,
    epochs=epochs,
    callbacks=[checkpoint], #, early_stopping
    verbose=1,
    validation_data=val_set,
    validation_steps=val_num//val_batch_size,
    #validation_split=0.2,
    shuffle=True
)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

 conv2d_10 (Conv2D)             (None, 64, 64, 128)  73856       ['max_pooling2d_1[0][0]']        
                                                                                                  
 conv2d_12 (Conv2D)             (None, 64, 64, 128)  8320        ['max_pooling2d_1[0][0]']        
                                                                                                  
 conv2d_11 (Conv2D)             (None, 64, 64, 128)  147584      ['conv2d_10[0][0]']              
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 64, 64, 128)  512        ['conv2d_12[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 64, 64, 128)  512        ['conv2d_11[0][0]']              
 rmalizati

                                                                  'batch_normalization_15[0][0]'] 
                                                                                                  
 activation_8 (Activation)      (None, 32, 32, 256)  0           ['add_7[0][0]']                  
                                                                                                  
 conv2d_25 (Conv2D)             (None, 32, 32, 256)  590080      ['activation_8[0][0]']           
                                                                                                  
 conv2d_27 (Conv2D)             (None, 32, 32, 256)  65792       ['activation_8[0][0]']           
                                                                                                  
 conv2d_26 (Conv2D)             (None, 32, 32, 256)  590080      ['conv2d_25[0][0]']              
                                                                                                  
 batch_nor

                                                                                                  
 add_12 (Add)                   (None, 32, 32, 256)  0           ['batch_normalization_26[0][0]', 
                                                                  'batch_normalization_25[0][0]'] 
                                                                                                  
 activation_13 (Activation)     (None, 32, 32, 256)  0           ['add_12[0][0]']                 
                                                                                                  
 max_pooling2d_3 (MaxPooling2D)  (None, 16, 16, 256)  0          ['activation_13[0][0]']          
                                                                                                  
 conv2d_40 (Conv2D)             (None, 16, 16, 512)  1180160     ['max_pooling2d_3[0][0]']        
                                                                                                  
 conv2d_42

 batch_normalization_33 (BatchN  (None, 16, 16, 256)  1024       ['conv2d_53[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_21 (Activation)     (None, 16, 16, 256)  0           ['batch_normalization_33[0][0]'] 
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 32, 32, 256)  590080     ['activation_21[0][0]']          
 ose)                                                                                             
                                                                                                  
 batch_normalization_34 (BatchN  (None, 32, 32, 256)  1024       ['conv2d_transpose[0][0]']       
 ormalization)                                                                                    
          

                                )                                                                 
                                                                                                  
 conv2d_transpose_3 (Conv2DTran  (None, 256, 256, 64  36928      ['activation_30[0][0]']          
 spose)                         )                                                                 
                                                                                                  
 batch_normalization_43 (BatchN  (None, 256, 256, 64  256        ['conv2d_transpose_3[0][0]']     
 ormalization)                  )                                                                 
                                                                                                  
 activation_31 (Activation)     (None, 256, 256, 64  0           ['batch_normalization_43[0][0]'] 
                                )                                                                 
          

775/775 [==============================] - 4145s 5s/step - loss: 0.6897 - binary_accuracy: 0.5109 - val_loss: 0.6936 - val_binary_accuracy: 0.5000
Epoch 35/10000
775/775 [==============================] - 4185s 5s/step - loss: 0.6896 - binary_accuracy: 0.5109 - val_loss: 0.6936 - val_binary_accuracy: 0.5000
Epoch 36/10000
775/775 [==============================] - 4360s 6s/step - loss: 0.6896 - binary_accuracy: 0.5109 - val_loss: 0.6936 - val_binary_accuracy: 0.5000
Epoch 37/10000
775/775 [==============================] - 4326s 6s/step - loss: 0.6895 - binary_accuracy: 0.5109 - val_loss: 0.6936 - val_binary_accuracy: 0.5000
Epoch 38/10000
775/775 [==============================] - 4337s 6s/step - loss: 0.6893 - binary_accuracy: 0.5109 - val_loss: 0.6936 - val_binary_accuracy: 0.5000
Epoch 39/10000
775/775 [==============================] - 4361s 6s/step - loss: 0.6893 - binary_accuracy: 0.5109 - val_loss: 0.6936 - val_binary_accuracy: 0.5000
Epoch 40/10000
775/775 [===================